#Creación interfaz web

## Instalar las Bibliotecas Necesarias

In [2]:
# Instalar dependencias
!pip install streamlit
!pip install pyngrok
!pip install h5py
!pip install wfdb
!pip install scipy

## Configurar ngrok

In [3]:
# Configurar el token de autenticación de ngrok
from pyngrok import ngrok
ngrok.set_auth_token("2hyfIhQEDNa5vywJs29AZMxgzCt_88A11MpMVyNtS1ZCPjk6Y")

# Cerrar túneles existentes
ngrok.kill()

## Creación de interfaz web

In [4]:
# Crear el archivo de la aplicación Streamlit
app_code = """
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import wfdb
import h5py
import scipy.io
import os
from scipy.signal import butter, lfilter

# Configurar estilos
st.set_page_config(page_title="Detección y clasificación de arritmia", page_icon=":heartbeat:")

st.markdown(
    \"\"\"
    <style>
    .reportview-container {
        background: #f2f2f2;
    }
    .sidebar .sidebar-content {
        background: #f2f2f2;
    }
    .stButton>button {
        background-color: #4CAF50;
        color: white;
    }
    .stTextInput>div>div>input {
        color: black;
    }
    </style>
    \"\"\",
    unsafe_allow_html=True
)

# Funciones de preprocesamiento
def z_score_normalization(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    return (signal - mean) / std

def bandpass_filter(signal, lowcut, highcut, fs, order=1):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, signal)

def derivative(signal):
    b = [1, 2, 0, -2, -1]
    a = 8
    return lfilter(b, a, signal)

def squaring(signal):
    return signal ** 2

def moving_average(signal, window_size):
    return np.convolve(signal, np.ones(window_size)/window_size, mode='same')

def detect_r_peaks(signal, fs):
    filtered_signal = bandpass_filter(signal, 5, 15, fs, order=1)
    derivative_signal = derivative(filtered_signal)
    squared_signal = squaring(derivative_signal)
    window_size = int(0.12 * fs)
    averaged_signal = moving_average(squared_signal, window_size)
    peaks = []
    threshold = np.mean(averaged_signal)
    for i in range(1, len(averaged_signal) - 1):
        if averaged_signal[i] > threshold and averaged_signal[i] > averaged_signal[i - 1] and averaged_signal[i] > averaged_signal[i + 1]:
            peaks.append(i)
    return np.array(peaks)

def segment_signal(signal, r_peaks, segment_length=360):
    segments = []
    half_length = segment_length // 2
    for r in r_peaks:
        if r - half_length >= 0 and r + half_length < len(signal):
            segments.append(signal[r - half_length:r + half_length])
    return np.array(segments)

# Función para preprocesar los datos
def preprocess_signal(signal, fs=250):
    normalized_signal = z_score_normalization(signal)
    r_peaks = detect_r_peaks(normalized_signal, fs)
    segments = segment_signal(normalized_signal, r_peaks)
    valid_segments = [seg for seg in segments if len(seg) == 360]
    valid_segments = np.array(valid_segments).reshape(-1, 360, 1)
    return valid_segments

# Función para extraer el índice del canal II
def extract_channel_index(hea_content, channel_name="II"):
    for i, line in enumerate(hea_content.splitlines()):
        if channel_name in line:
            return i
    return None

# Función para leer archivos .hea y .mat, y extraer el canal II
def process_files(hea_file, mat_file, channel_name="II"):
    try:
        with open(hea_file, 'r') as f:
            hea_content = f.read()
    except Exception as e:
        raise ValueError(f"Error al leer el archivo .hea: {e}")

    try:
        mat_content = scipy.io.loadmat(mat_file)
        mat_data = mat_content['val']
    except NotImplementedError:
        try:
            mat_content = h5py.File(mat_file, 'r')
            mat_data = np.array(mat_content['val'])
        except Exception as e:
            raise ValueError(f"Error al leer el archivo .mat: {e}")

    channel_index = extract_channel_index(hea_content, channel_name)

    if channel_index is not None:
        signal = mat_data[channel_index]
        labels = [line.strip() for line in hea_content.splitlines() if line.startswith('#')]
        return signal, labels
    else:
        raise ValueError(f"Canal {channel_name} no encontrado en el archivo .hea")

# Cargar tu modelo guardado
model = load_model('/content/modelo_red_rnn_ajustado.h5')  # Reemplaza 'path_to_your_model.h5' con la ruta real a tu archivo .h5

# Función para interpretar predicciones
def interpretar_predicciones(alarm_predictions, arrhythmia_predictions):
    alarm_labels = ['Falsa alarma', 'Alarma verdadera']
    arrhythmia_labels = ['Asistolia', 'Bradicardia', 'Taquicardia', 'Flutter/Fibrilación ventricular', 'Taquicardia ventricular']

    alarm_results = [alarm_labels[np.argmax(pred)] for pred in alarm_predictions]
    arrhythmia_results = [arrhythmia_labels[np.argmax(pred)] for pred in arrhythmia_predictions]

    return alarm_results, arrhythmia_results

# Función para traducir las etiquetas reales del archivo .hea
def traducir_etiquetas(labels):
    traduccion = {
        '#False alarm': 'Falsa alarma',
        '#True alarm': 'Alarma verdadera',
        '#Asystole': 'Asistolia',
        '#Bradycardia': 'Bradicardia',
        '#Tachycardia': 'Taquicardia',
        '#Ventricular_Flutter_Fib': 'Flutter/Fibrilación ventricular',
        '#Ventricular_Tachycardia': 'Taquicardia ventricular'
    }
    return [traduccion.get(label, label) for label in labels]

# Definir una función para hacer predicciones
def predict(data):
    predictions = model.predict(data)
    alarm_predictions = predictions[0]
    arrhythmia_predictions = predictions[1]

    return interpretar_predicciones(alarm_predictions, arrhythmia_predictions)

# Crear la interfaz web con Streamlit
st.title("Detección y clasificación de arritmia")

uploaded_hea_file = st.file_uploader("Sube tu archivo .hea", type=["hea"])
uploaded_mat_file = st.file_uploader("Sube tu archivo .mat", type=["mat"])

if uploaded_hea_file is not None and uploaded_mat_file is not None:
    # Guardar los archivos subidos
    with open("uploaded.hea", "wb") as f:
        f.write(uploaded_hea_file.getbuffer())
    with open("uploaded.mat", "wb") as f:
        f.write(uploaded_mat_file.getbuffer())

    # Procesar los archivos y extraer la señal del canal II y las etiquetas
    try:
        st.write("Leyendo archivos...")
        ecg_signal, real_labels = process_files("uploaded.hea", "uploaded.mat")
        st.write("Archivos leídos correctamente.")

        # Preprocesar la señal
        st.write("Preprocesando señal...")
        preprocessed_data = preprocess_signal(ecg_signal)
        st.write("Señal preprocesada correctamente.")

        # Guardar la señal preprocesada en un archivo .npy
        np.save('preprocessed_data.npy', {'signal': preprocessed_data})

        # Cargar la señal preprocesada desde el archivo .npy
        data = np.load('preprocessed_data.npy', allow_pickle=True).item()
        ecg_signal = data['signal']


        # Realizar predicciones
        alarm_results, arrhythmia_results = predict(ecg_signal)

        # Traducir y mostrar etiquetas reales
        st.subheader("*Etiquetas reales de la señal*")
        etiquetas_traducidas = traducir_etiquetas(real_labels)
        for label in etiquetas_traducidas:
            st.write(f"*{label}*")

        # Mostrar solo un resultado de alarma y un resultado de arritmia
        st.subheader("*Resultados de predicción*")
        st.write(f"*{arrhythmia_results[0]}*")
        st.write(f"*{alarm_results[0]}*")


    except Exception as e:
        st.error(f"Error al procesar los archivos: {e}")
"""

## Guardar y publicar interfaz

In [5]:
# Escribir el código en un archivo
with open('app.py', 'w') as f:
    f.write(app_code)

# Iniciar Streamlit y ngrok
import os

# Iniciar Streamlit en segundo plano
os.system('streamlit run app.py &')

# Crear un túnel utilizando ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit está disponible en: {public_url}")

Streamlit está disponible en: NgrokTunnel: "https://90bb-34-16-145-107.ngrok-free.app" -> "http://localhost:8501"
